In [1]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns
import pickle

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm.notebook import tqdm
from notebook import notebookapp
import urllib
import json
import ipykernel
from pathlib import Path

from scipy.stats import norm
# from sklearn.naive_bayes import ComplementNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.kernel_ridge import KernelRidge
# from scipy.stats import ks_2samp
import pickle
import gc


import warnings
warnings.filterwarnings('ignore')

import bz2
import pickle
import _pickle as cPickle

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

TEST_PRIVATE = False
TEST_LOCAL = False

### Functions

In [2]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

def log_loss_score(actual, predicted,  eps=1e-15):

        """
        :param predicted:   The predicted probabilities as floats between 0-1
        :param actual:      The binary labels. Either 0 or 1.
        :param eps:         Log(0) is equal to infinity, so we need to offset our predicted values slightly by eps from 0 or 1
        :return:            The logarithmic loss between between the predicted probability assigned to the possible outcomes for item i, and the actual outcome.
        """
        
        p1 = actual * np.log(predicted+eps)
        p0 = (1-actual) * np.log(1-predicted+eps)
        loss = p0 + p1

        return -loss.mean()
    
def log_loss_multi(y_true, y_pred):
    M = y_true.shape[1]
    results = np.zeros(M)
    for i in range(M):
        results[i] = log_loss_score(y_true[:,i], y_pred[:,i])
    return results.mean()

### DataBase

In [3]:
class CFG:
    num_folds = 10
    use_var_enc = True
    variance_thres = 0.60
    quantile_transform = True
    use_rankgauss = True
    
    selec_top = True
    original_feats = True #False remove also top features
    
    use_pca = True
    pca_comp_genes = 29 #30
    pca_comp_cells = 4 #18
    
    use_fastica = False
    fastica_comp_genes = 10
    fastica_comp_cells = 5
    
    use_kridge = True
    use_xgb = False
    use_bayes = False


### Load Databases

In [4]:
seed_everything(seed=42)

train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
train_drug = pd.read_csv('../input/lish-moa/train_drug.csv')

drug_dict = dict(zip(train_drug['sig_id'],train_drug['drug_id']))

test_features = pd.read_csv('../input/lish-moa/test_features.csv')

if TEST_PRIVATE:
    id1 = [test_features['sig_id'].values]
    for other_index in ['A', 'B', 'C']:
        id1.append(id1[0]+other_index)
    test_features = pd.concat([test_features, test_features, test_features, test_features],sort=False).reset_index(drop=True)
    test_features['sig_id'] = np.concatenate(id1)
    print(test_features.shape)

sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

train_orig = train_features.merge(train_targets_scored, on='sig_id')
train_orig = train_orig.merge(train_drug, on='sig_id', how='left')

train_noctl = train_orig[train_orig['cp_type']!='ctl_vehicle'].drop('cp_type', axis=1).reset_index(drop=True)
train_ctl = train_orig[train_orig['cp_type']=='ctl_vehicle'].drop('cp_type', axis=1).reset_index(drop=True)

test_noctl = test_features[test_features['cp_type']!='ctl_vehicle'].drop('cp_type', axis=1).reset_index(drop=True)
test_ctl = test_features[test_features['cp_type']=='ctl_vehicle'].drop('cp_type', axis=1).reset_index(drop=True)

target = train_noctl[train_targets_scored.columns]

GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()
other_feats = []

# folds = train_noctl.copy()
# mskf = MultilabelStratifiedKFold(n_splits=CFG.num_folds, shuffle=True, random_state=43)

# for f, (t_idx, v_idx) in enumerate(mskf.split(X=train_noctl, y=target)):
#     folds.loc[v_idx, 'kfold'] = int(f)

# folds['kfold'] = folds['kfold'].astype(int)

folds = train_noctl.copy()

print(train_orig.shape)
print(target.shape)
print(sample_submission.shape)
print(folds.shape, test_noctl.shape)

(23814, 1083)
(21948, 207)
(3982, 207)
(21948, 1082) (3624, 875)


### Create Kridge Features

In [5]:
min_num_ones = 20
folds_kridge = pd.read_csv('../input/kridgefeats25oct/folds.csv')
test_kridge = test_noctl[['sig_id']].copy()

colsk_selec = []
min_num_ones = 20
for ncol, col_target in enumerate(tqdm(target_cols)):
    name_feat = f'kridge_{col_target}'
    if np.sum(folds[col_target].sum())>=min_num_ones:
        colsk_selec.append(name_feat)
        # Save all
        with bz2.BZ2File(f'../input/kridgefeats25oct/feats__{name_feat}.pbz2','rb') as file:
            model = cPickle.load(file)
            feats_trn_uncorr = cPickle.load(file)
            
#         with open(f'../input/kridgefeats/model__{name_feat}__{col_target}.pkl', 'rb') as file:
#             model = pickle.load(file)
#             feats_trn_uncorr = pickle.load(file)

        # With Test DB
        X_trn = folds[feats_trn_uncorr].astype(float).values
        X_test = test_noctl[feats_trn_uncorr].astype(float).values
        y = folds[col_target].astype(float).values 

        scaler = StandardScaler()
        X_trn = scaler.fit_transform(X_trn)
        X_test = scaler.transform(X_test)
        
        # Training matrix has been removed to save memory
        model.X_fit_ = X_trn
    #     model = KernelRidge(alpha = alpha, kernel = 'rbf')
    #     model.fit(X_trn, y)
        test_kridge[name_feat] = model.predict(X_test)

kridge_cols = ['sig_id']+colsk_selec
print(folds_kridge[kridge_cols].shape, test_kridge[kridge_cols].shape)

# Comprueba con original
if TEST_LOCAL and not TEST_PRIVATE:
    test_feat_orig = pd.read_csv('../input/kridgefeats25oct/test.csv')
    print(np.sum(test_feat_orig[colsk_selec].values - test_kridge[colsk_selec].values))
    del test_feat_orig
    gc.collect()


(21948, 154) (3624, 154)


In [6]:
# min_num_ones = 20
# folds_kridge = pd.read_csv('../input/kridgefeats/folds.csv')
# test_kridge = test_noctl['sig_id'].copy()

# colsk_selec = []
# min_num_ones = 20
# for ncol, col_target in enumerate(tqdm(target_cols)):
#     name_feat = f'kridge_{col_target}'
#     if np.sum(folds[col_target].sum())>=min_num_ones:
#         colsk_selec.append(name_feat)
#         # Save all
#         with open(f'../input/kridgefeats/model__{name_feat}__{col_target}.pkl', 'rb') as file:
#             model = pickle.load(file)
#             feats_trn_uncorr = pickle.load(file)

#         # With Test DB
#         X_trn = folds[feats_trn_uncorr].astype(float).values
#         X_test = test_noctl[feats_trn_uncorr].astype(float).values
#         y = folds[col_target].astype(float).values 

#         scaler = StandardScaler()
#         X_trn = scaler.fit_transform(X_trn)
#         X_test = scaler.transform(X_test)

#     #     model = KernelRidge(alpha = alpha, kernel = 'rbf')
#     #     model.fit(X_trn, y)
#         test_kridge[name_feat] = model.predict(X_test)

# kridge_cols = ['sig_id']+colsk_selec
# print(folds_kridge[kridge_cols].shape, test_kridge[kridge_cols].shape)

# # Comprueba con original
# if False:
#     test_feat_orig = pd.read_csv('../input/kridgefeats/test.csv')
#     print(np.sum(test_feat_orig[colsk_selec].values - test_kridge[colsk_selec].values))
#     del test_feat_orig
#     gc.collect()

In [7]:
# Create KRIDGE Features
# def trimm_correlated(df_in, threshold):
#     df_corr = df_in.corr(method='pearson', min_periods=1)
#     df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
#     un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
#     return un_corr_idx

# from scipy.stats import norm
# from sklearn.naive_bayes import ComplementNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_auc_score
# from sklearn.preprocessing import MinMaxScaler, StandardScaler
# from sklearn.kernel_ridge import KernelRidge
# from scipy.stats import ks_2samp
# import pickle

# folds_feat = folds.copy()
# test_feat = test_noctl.copy()

# folds_feat['cp_dose'] = (folds_feat['cp_dose']=='D2').astype(float)
# folds_feat['cp_time'] /= 72.0

# test_feat['cp_dose'] = (test_feat['cp_dose']=='D2').astype(float)
# test_feat['cp_time'] /= 72.0

# # C_logistic = 0.01
# thr= 0.05 #p-value thr
# thr2 = 0.93 #Correlation Thr

# # for thr in [0.05]: #1, 0.03, 0.05, 0.07, 0.09]:
# #     for thr2 in [0.90, 0.93, 0.95, 0.97, 1.00]:
# res = []
# res_best = []
# features_models = []
# list_target_cols = []
# list_new_cols = []
# scaler = StandardScaler()
# list_AUC = []
# list_loss = []

# for ncol, col_target in enumerate(tqdm(target_cols)):
# #             if ncol<=50: #00:
#     list_target_cols.append(col_target)
#     name_feat = f'kridge_{col_target}'
#     list_new_cols.append(name_feat)

#     # Compare distribution of positives and negatives with kolgomorov
#     pvalues = []
#     for col_feat in GENES+CELLS:
#         values_orig = train_noctl[col_feat].values
#         target_orig = train_noctl[col_target].values
#         target = target_orig[values_orig> -4]
#         values = values_orig[values_orig> -4]
#         positive = values[target==1]
#         negative = values[target==0]
#         if len(positive)>0:
#             pvalue = ks_2samp(positive, negative)[1]
#         else:
#             pvalue = 1.0
#         pvalues.append(pvalue)
#     pvalues = np.array(pvalues)

#     # Search best model with different thresholds
#     best_auc = 0.0
#     best_loss = 999999.9
#     best_thr = 0.0 # thr=pvalue
#     best_thr2 = 0.0 #th2=correlation
#     best_lenfeats = 0
#     best_lenfeats_uncorr = 0
#     best_list_feats = []
#     alpha = 100
#     folds_feat[name_feat] = 0.0
#     feats_trn = np.array(GENES+CELLS)[pvalues<thr]
#     feats_trn_uncorr = trimm_correlated(folds_feat[feats_trn], thr2).tolist()
#     if len(feats_trn_uncorr)>0:
#         X_trn = folds_feat.loc[:, feats_trn_uncorr].astype(float).values
#         X_trn = scaler.fit_transform(X_trn)

#         for fold in range(CFG.num_folds):
#             X_trn_fold = X_trn[folds_feat['kfold']!=fold, :]
#             X_val_fold = X_trn[folds_feat['kfold']==fold, :]
#             y_trn_fold = folds_feat.loc[folds_feat['kfold']!=fold, col_target].astype(float).values
#             if np.sum(y_trn_fold)>10:
#                 model = KernelRidge(alpha = alpha, kernel = 'rbf')
# #                 model = LogisticRegression(C=C_logistic)
#                 model.fit(X_trn_fold, y_trn_fold)
# #                 folds_feat.loc[folds_feat['kfold']==fold, name_feat] = model.predict_proba(X_val_fold)[:,1]
#                 folds_feat.loc[folds_feat['kfold']==fold, name_feat] = model.predict(X_val_fold)

#         auc = roc_auc_score(folds_feat[col_target].values, folds_feat[name_feat].values)
#         list_AUC.append(auc)
#         loss = log_loss(folds_feat[col_target].values, folds_feat[name_feat].values)
#         list_loss.append(loss)
#         lenfeats = len(feats_trn)
#         lenfeats_uncorr = len(feats_trn_uncorr)
#         res.append(dict({'ncol':ncol, 'col':col_target, 'thr':thr, 'thr2':thr2, \
#                          'lenfeats':lenfeats, 'lenfeats_uncorr':lenfeats_uncorr, \
#                          'sumones':folds_feat[col_target].sum(), 'AUC':auc, 'Mean_AUC':np.mean(list_AUC), \
#                          'loss':loss, 'Mean_loss':np.mean(list_loss)}))
# #                         print(f'{ncol+1}/{len(target_cols)}: col={col_target} thr={thr} thr2={thr2} lenfeats={lenfeats} lenfeats_uncorr={lenfeats_uncorr} sum_ones={folds_feat[col_target].sum()} loss={loss} AUC={auc} Mean_AUC={np.mean(list_AUC)}')
#         if loss < best_loss:
#             best_auc = auc
#             best_loss = loss
#             best_thr = thr
#             best_thr2 = thr2
#             best_lenfeats = lenfeats
#             best_lenfeats_uncorr = lenfeats_uncorr
#             preds = folds_feat[name_feat].values
#             best_list_feats = feats_trn_uncorr

#         # Get best
#         loss = best_loss
#         auc = best_auc
#         thr = best_thr
#         thr2 = best_thr2
#         lenfeats = best_lenfeats
#         lenfeats_uncorr = best_lenfeats_uncorr
#         folds_feat[name_feat] = preds
#         feats_trn_uncorr = best_list_feats

#         # Provisional mlogloss
#         valid_results_score = train_targets_scored[['sig_id']].merge(folds_feat[['sig_id']+list_new_cols], on='sig_id', how='left').fillna(0)
#         y_true = train_targets_scored[list_target_cols].values
#         valid_results_score.columns = ['sig_id']+list_target_cols
#         y_pred = valid_results_score[list_target_cols].values
#         score = 0
#         for i in range(len(list_target_cols)):
#             score_ = log_loss(y_true[:, i], y_pred[:, i])
#             score += score_ 
#         score /= len(list_target_cols)
# #         print(C_logistic, thr, thr2, np.mean(list_AUC))
#         print(f'BEST {ncol+1}/{len(target_cols)}: col={col_target} thr={thr} thr2={thr2} lenfeats={lenfeats} lenfeats_uncorr={lenfeats_uncorr} sum_ones={folds_feat[col_target].sum()} loss={loss} AUC={auc} Mean_AUC={np.mean(list_AUC)} mlogloss={score}')
        
#         # With Test DB
#         X_trn = folds_feat[feats_trn_uncorr].astype(float).values
#         X_test = test_feat[feats_trn_uncorr].astype(float).values
#         y = folds_feat[col_target].astype(float).values 

#         scaler = StandardScaler()
#         X_trn = scaler.fit_transform(X_trn)
#         X_test = scaler.transform(X_test)

#         model = KernelRidge(alpha = alpha, kernel = 'rbf')
# #         model = LogisticRegression(C=C_logistic)
#         model.fit(X_trn, y)
# #         test_feat[name_feat] = model.predict_proba(X_test)[:,1]
#         test_feat[name_feat] = model.predict(X_test)

#         # Save all
#         with open(output_dir / f'model__{name_feat}__{col_target}.pkl', 'wb') as file:
#             pickle.dump(model, file)
#             pickle.dump(feats_trn_uncorr, file)
        
#         folds_feat.to_csv( output_dir / 'folds.csv',index=False)
#         test_feat.to_csv(output_dir / 'test.csv',index=False)
        
#         res_best.append(dict({'ncol':ncol, 'col':col_target, 'thr':thr, 'thr2':thr2, \
#                      'lenfeats':lenfeats, 'lenfeats_uncorr':lenfeats_uncorr, \
#                      'sumones':folds_feat[col_target].sum(), 'AUC':auc, 'loss':loss, 'mlogloss':score}))
#         res_df = pd.DataFrame(res_best)
#         res_df.to_csv(output_dir / 'res.csv',index=False)

### RankGauss

In [8]:
#From: https://www.kaggle.com/kushal1506/moa-pytorch-0-01859-rankgauss-pca-nn
folds_cp = folds.copy()
test_noctl_cp = test_noctl.copy()

if CFG.use_rankgauss:
    for col in (GENES + CELLS):
        transformer = QuantileTransformer(n_quantiles=100,random_state=42, output_distribution="normal")
        vec_len = len(folds_cp[col].values)
        vec_len_test = len(test_noctl_cp[col].values)
        raw_vec = folds_cp[col].values.reshape(vec_len, 1)
        transformer.fit(raw_vec)

        folds_cp[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
        test_noctl_cp[col] = transformer.transform(test_noctl_cp[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

### KMeans Features

In [9]:
from sklearn.cluster import KMeans
import gc

def create_cluster(train, test, features, kind = 'g', n_clusters = 35):
    train_ = train[features].copy()
    test_ = test[features].copy()
    
    scaler = StandardScaler()
    train_ = scaler.fit_transform(train_)
    test_ = scaler.transform(test_)
    model = KMeans(n_clusters = n_clusters, random_state = 123).fit(train_)
    name_cols = [f'kmeans_{kind}_{i}' for i in range(n_clusters)]
    train = pd.concat([train, pd.DataFrame(model.transform(train_), columns=name_cols)], axis=1)
    test = pd.concat([test, pd.DataFrame(model.transform(test_), columns=name_cols)], axis=1)
    print(train.shape, test.shape)
    return train, test
    
folds_cp2 = folds.copy()
test_noctl_cp2 = test_noctl.copy()
folds_cp2, test_noctl_cp2 = create_cluster(folds_cp2,test_noctl_cp2, GENES, kind = 'g', n_clusters = 7)
folds_cp2, test_noctl_cp2 = create_cluster(folds_cp2,test_noctl_cp2, CELLS, kind = 'c', n_clusters = 7)

folds_3models = folds_cp2[list(folds_cp2.columns.values[np.where(folds_cp2.columns=='kmeans_g_0')[0][0]:])]
test_noctl_3model = test_noctl_cp2[list(folds_cp2.columns.values[np.where(folds_cp2.columns=='kmeans_g_0')[0][0]:])]
del folds_cp2, test_noctl_cp2
gc.collect()

# train_features ,test_features=fe_cluster(train_features,test_features)

(21948, 1089) (3624, 882)
(21948, 1096) (3624, 889)


12

### PCA Features

In [10]:
if CFG.use_pca:
    seed_everything(seed=42)
    etiq = ['G','C']
    num_pca = [CFG.pca_comp_genes, CFG.pca_comp_cells]
    for niter, cols in enumerate([GENES, CELLS]):
        # PCA for train with folds
        num_comp = num_pca[niter]
        columns_pca = [f'pca_{etiq[niter]}-{i}' for i in range(num_comp)]
        other_feats += columns_pca

        # PCA for train
        pca = PCA(n_components=num_comp, random_state=42).fit(folds_cp[cols])
        train_pca = pca.transform(folds_cp[cols])
        train_pca = pd.DataFrame(train_pca, columns=columns_pca)
        folds = pd.concat((folds, train_pca), axis=1)
        
        # PCA for train
        test_pca = pca.transform(test_noctl_cp[cols])
        test_pca = pd.DataFrame(test_pca, columns=columns_pca)
        test_noctl = pd.concat((test_noctl, test_pca), axis=1)
        print(folds.shape, test_noctl.shape)

(21948, 1111) (3624, 904)
(21948, 1115) (3624, 908)


In [11]:
# if CFG.use_pca:
#     seed_everything(seed=42)
#     etiq = ['G','C']
#     num_pca = [CFG.pca_comp_genes, CFG.pca_comp_cells]
#     for niter, cols in enumerate([GENES, CELLS]):
#         # PCA for train with folds
#         train_pca = []
#         train_pca_sig_id = []
#         num_comp = num_pca[niter]
#         columns_pca = [f'pca_{etiq[niter]}-{i}' for i in range(num_comp)]
#         other_feats += columns_pca
#         for fold in tqdm(range(CFG.num_folds)):
#             pca = PCA(n_components=num_comp, random_state=42).fit(folds.loc[folds['kfold']!=fold, cols])
#             train_pca.append(pca.transform(folds.loc[folds['kfold']==fold, cols]))
#             train_pca_sig_id.append(folds.loc[folds['kfold']==fold, 'sig_id'])
#         train_pca = np.concatenate(train_pca)
#         train_pca = pd.DataFrame(train_pca, columns=columns_pca)
#         train_pca['sig_id'] = np.concatenate(train_pca_sig_id)
#         folds = pd.merge(folds, train_pca, on='sig_id')

#         # PCA for test
#         pca = PCA(n_components=num_comp, random_state=42).fit(folds[cols])
#         test_pca = pca.transform(test_noctl[cols])
#         test_pca = pd.DataFrame(test_pca, columns=columns_pca)
#         test_noctl = pd.concat((test_noctl, test_pca), axis=1)
#         print(folds.shape, test_noctl.shape)

### Select Top Features

In [12]:
# from sklearn.utils import check_random_state  # type: ignore

# ### from eli5
# def iter_shuffled(X, columns_to_shuffle=None, pre_shuffle=False, random_state=None):
#     rng = check_random_state(random_state)

#     if columns_to_shuffle is None:
#         columns_to_shuffle = range(X.shape[1])

#     if pre_shuffle:
#         X_shuffled = X.copy()
#         rng.shuffle(X_shuffled)

#     X_res = X.copy()
#     for columns in tqdm(columns_to_shuffle):
#         if pre_shuffle:
#             X_res[:, columns] = X_shuffled[:, columns]
#         else:
#             rng.shuffle(X_res[:, columns])
#         yield X_res
#         X_res[:, columns] = X[:, columns]

# def get_score_importances(
#         score_func,  # type: Callable[[Any, Any], float]
#         X,
#         y,
#         n_iter=5,  # type: int
#         columns_to_shuffle=None,
#         random_state=None
#     ):
#     rng = check_random_state(random_state)
#     base_score = score_func(X, y)
#     scores_decreases = []
#     for i in range(n_iter):
#         scores_shuffled = _get_scores_shufled(
#             score_func, X, y, columns_to_shuffle=columns_to_shuffle,
#             random_state=rng, base_score=base_score
#         )
#         scores_decreases.append(scores_shuffled)

#     return base_score, scores_decreases

# def _get_scores_shufled(score_func, X, y, base_score, columns_to_shuffle=None,
#                         random_state=None):
#     Xs = iter_shuffled(X, columns_to_shuffle, random_state=random_state)
#     res = []
#     for X_shuffled in Xs:
#         res.append(-score_func(X_shuffled, y) + base_score)
#     return res

# def metric(y_true, y_pred):
#     metrics = []
#     for i in range(y_pred.shape[1]):
#         if y_true[:, i].sum() > 1:
#             metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float)))
#     return np.mean(metrics)   

# perm_imp = np.zeros(train.shape[1])
# all_res = []
# for n, (tr, te) in enumerate(KFold(n_splits=7, random_state=0, shuffle=True).split(train_targets)):
#     print(f'Fold {n}')

#     model = create_model(len(train.columns))
#     checkpoint_path = f'repeat:{seed}_Fold:{n}.hdf5'
#     reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
#     cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True,
#                                      save_weights_only = True, mode = 'min')
#     model.fit(train.values[tr],
#                   train_targets.values[tr],
#                   validation_data=(train.values[te], train_targets.values[te]),
#                   epochs=35, batch_size=128,
#                   callbacks=[reduce_lr_loss, cb_checkpt], verbose=2
#                  )
        
#     model.load_weights(checkpoint_path)
        
#     def _score(X, y):
#         pred = model.predict(X)
#         return metric(y, pred)

#     base_score, local_imp = get_score_importances(_score, train.values[te], train_targets.values[te], n_iter=1, random_state=0)
#     all_res.append(local_imp)
#     perm_imp += np.mean(local_imp, axis=0)
#     print('')
    
# top_feats = np.argwhere(perm_imp < 0).flatten()
# top_feats


In [13]:
if CFG.selec_top:
    seed_everything(seed=42)
    # https://www.kaggle.com/simakov/keras-multilabel-neural-network-v1-2/data
    top_feats = [  1,   2,   3,   4,   5,   6,   7,   9,  11,  14,  15,  16,  17,
            18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,  31,
            32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
            47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,  60,
            61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
            74,  75,  76,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
            89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
           102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
           115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128,
           129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143,
           144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157,
           158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
           171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
           184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197,
           198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212,
           213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226,
           227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
           240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
           254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266,
           267, 268, 269, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
           281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294,
           295, 296, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,
           310, 311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323,
           324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336,
           337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
           350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362,
           363, 364, 365, 366, 367, 368, 369, 370, 371, 374, 375, 376, 377,
           378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 391,
           392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404,
           405, 406, 407, 408, 409, 411, 412, 413, 414, 415, 416, 417, 418,
           419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
           432, 434, 435, 436, 437, 438, 439, 440, 442, 443, 444, 445, 446,
           447, 448, 449, 450, 453, 454, 456, 457, 458, 459, 460, 461, 462,
           463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
           476, 477, 478, 479, 481, 482, 483, 484, 485, 486, 487, 488, 489,
           490, 491, 492, 493, 494, 495, 496, 498, 500, 501, 502, 503, 505,
           506, 507, 509, 510, 511, 512, 513, 514, 515, 518, 519, 520, 521,
           522, 523, 524, 525, 526, 527, 528, 530, 531, 532, 534, 535, 536,
           538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 549, 550, 551,
           552, 554, 557, 559, 560, 561, 562, 565, 566, 567, 568, 569, 570,
           571, 572, 573, 574, 575, 577, 578, 580, 581, 582, 583, 584, 585,
           586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599,
           600, 601, 602, 606, 607, 608, 609, 611, 612, 613, 615, 616, 617,
           618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630,
           631, 632, 633, 634, 635, 636, 637, 638, 639, 641, 642, 643, 644,
           645, 646, 647, 648, 649, 650, 651, 652, 654, 655, 656, 658, 659,
           660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672,
           673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685,
           686, 687, 688, 689, 691, 692, 693, 694, 695, 696, 697, 699, 700,
           701, 702, 704, 705, 707, 708, 709, 710, 711, 713, 714, 716, 717,
           718, 720, 721, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732,
           733, 734, 735, 737, 738, 739, 740, 742, 743, 744, 745, 746, 747,
           748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 759, 760, 761,
           762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774,
           775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788,
           789, 790, 792, 793, 794, 795, 796, 797, 798, 800, 801, 802, 803,
           804, 805, 806, 808, 809, 811, 813, 814, 815, 816, 817, 818, 819,
           821, 822, 823, 825, 826, 827, 828, 829, 830, 831, 832, 834, 835,
           837, 838, 839, 840, 841, 842, 845, 846, 847, 848, 850, 851, 852,
           854, 855, 856, 858, 859, 860, 861, 862, 864, 866, 867, 868, 869,
           870, 871, 872, 873, 874]
    # print(len(top_feats))
    selected_features = np.array(['cp_type','cp_time', 'cp_dose']+GENES+CELLS)[top_feats].tolist()
    GENES = [col for col in selected_features if 'g-' in col]
    CELLS = [col for col in selected_features if 'c-' in col]
    numeric_cols = other_feats + GENES + CELLS
    folds = folds[['sig_id','cp_time','cp_dose']+numeric_cols+target_cols]
    test_noctl = test_noctl[['sig_id','cp_time','cp_dose']+numeric_cols]
    print(folds.shape, test_noctl.shape)

(21948, 1025) (3624, 819)


In [14]:
folds_novar_enc = folds.copy()
test_noctl_novar_enc = test_noctl.copy()

### Reduce Dataset using Variance

In [15]:
class VarianceThreshold:
    def __init__(self, threshold):
        self.threshold = threshold
    def fit(self, df, cont_cols):
        self.cont_cols = cont_cols
        self.var = folds[cont_cols].var()
        good_cols = self.var[self.var > self.threshold]
        self.index = good_cols.index.to_list()
        self.dropcols = [x for x in cont_cols if x not in self.var[self.var > self.threshold].index.to_list()]
        self.validcols = [x for x in cont_cols if x in self.var[self.var > self.threshold].index.to_list()]
    def transform(self, df):
        return df.drop(self.dropcols, axis=1)
    def fit_transform(self, df, cont_cols):
        self.fit(df, cont_cols)
        return self.transform(df), self.validcols

In [16]:
if CFG.use_var_enc:
    cont_cols_ini = [i for i in test_noctl.columns if i not in ['sig_id', 'cp_time', 'cp_dose']]
    print('Variance Threshold:', CFG.variance_thres)
    print(folds.shape, test_noctl.shape)
    VarThres = VarianceThreshold(CFG.variance_thres)
    folds, cont_cols = VarThres.fit_transform(folds, cont_cols_ini)
    test_noctl = VarThres.transform(test_noctl)
    print(folds.shape, test_noctl.shape)

Variance Threshold: 0.6
(21948, 1025) (3624, 819)
(21948, 1011) (3624, 805)


### Include kridge features  removing with low num of ones

In [17]:
if CFG.use_kridge:
    folds = pd.merge(folds, folds_kridge[kridge_cols], on='sig_id')
    test_noctl = pd.merge(test_noctl, test_kridge[kridge_cols], on='sig_id')
    print(folds.shape, test_noctl.shape)
    
    folds_novar_enc = pd.merge(folds_novar_enc, folds_kridge[kridge_cols], on='sig_id')
    test_noctl_novar_enc =  pd.merge(test_noctl_novar_enc, test_kridge[kridge_cols], on='sig_id')
    
    del folds_kridge
    del test_kridge
    gc.collect()
    
# #     seed_everything(seed=42)
# #     folds_feat = pd.read_csv('../input/folds_feats.csv')
# #     test_feat = pd.read_csv('../input/test_feat.csv')
# #     kridge_cols = ['sig_id']+[col for col in folds_feat if 'kridge' in col]
# #     folds = pd.merge(folds, folds_feat[kridge_cols], on='sig_id')
# #     test_noctl = pd.merge(test_noctl, test_feat[kridge_cols], on='sig_id')
# #     print(folds.shape, test_noctl.shape)
    
#     min_num_ones = 20
#     folds_feat = pd.read_csv('../results/_29_Create_KRIDGE_feats_16oct_e30/folds.csv')
#     test_feat = pd.read_csv('../results/_29_Create_KRIDGE_feats_16oct_e30/test.csv')
    
#     colsk = [col.replace('kridge_','') for col in folds_feat if 'kridge' in col]
#     colsk_selec = []
#     target_selec = []
#     for colk in colsk:
#         if np.sum(folds[colk].sum())>=min_num_ones:
#             colsk_selec += ['kridge_'+colk]
#             target_selec += [colk]
#     kridge_cols = ['sig_id']+colsk_selec

#     folds = pd.merge(folds, folds_feat[kridge_cols], on='sig_id')
#     test_noctl = pd.merge(test_noctl, test_feat[kridge_cols], on='sig_id')
#     print(len(colsk), len(colsk_selec), folds.shape, test_noctl.shape)
    
#     kridge_cols = [col for col in folds if 'kridge_' in col]
# #     print('logloss=',log_loss_multi(folds[target_selec].values, folds[kridge_cols].values))    

(21948, 1164) (3624, 958)


### Quantile Transform of Features


In [18]:
if CFG.quantile_transform:
    qt = QuantileTransformer(output_distribution='normal')
    folds[cont_cols] = qt.fit_transform(folds[cont_cols])
    test_noctl[cont_cols] = qt.transform(test_noctl[cont_cols])

# 1. ANN

### Training Functions

In [19]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

In [20]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [21]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    y_true = []
    y_pred = []
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        if not scheduler.__class__ ==  torch.optim.lr_scheduler.ReduceLROnPlateau:
            scheduler.step()
        
        final_loss += loss.item()
        y_true.append(targets.cpu().detach().numpy())
        y_pred.append(outputs.sigmoid().cpu().detach().numpy())
    
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    
    control_loss = log_loss_multi(y_true, y_pred)
    final_loss /= len(dataloader)
    return final_loss, control_loss

def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    y_true = []
    y_pred = []
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        y_true.append(targets.cpu().detach().numpy())
        y_pred.append(outputs.sigmoid().cpu().detach().numpy())
        
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    control_loss = log_loss_multi(y_true, y_pred)
    final_loss /= len(dataloader)
    return final_loss, control_loss, y_true, y_pred

def inference_fn(model, dataloader, device):
    model.eval()
    y_pred = []
    for data in dataloader:
        inputs = data['x'].to(device)
        with torch.no_grad():
            outputs = model(inputs)
        y_pred.append(outputs.sigmoid().detach().cpu().numpy())
    y_pred = np.concatenate(y_pred)
    
    return y_pred

### Model

In [22]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(drop1_feat) #0.20
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(drop2_feat) #0.20
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(drop3_feat) #0.25
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
#         x = F.relu(self.dense1(x))
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
#         x = F.relu(self.dense2(x))
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

### Add Kmeans Data

In [23]:
folds = pd.concat((folds, folds_3models), axis=1)
test_noctl = pd.concat((test_noctl, test_noctl_3model), axis=1)
print(folds.shape, test_noctl.shape)
del folds_3models, test_noctl_3model
gc.collect()

(21948, 1178) (3624, 972)


40

### Preprocessing Steps

In [24]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [25]:
folds = process_data(folds)
test_noctl = process_data(test_noctl)

feature_cols = [c for c in folds.columns if c not in target_cols]
if not CFG.original_feats: feature_cols = [c for c in feature_cols if c not in GENES+CELLS]
feature_cols_ini = [c for c in feature_cols if c not in ['kfold','sig_id']]#, 'cp_time_24', 'cp_time_48', 'cp_time_72', 'cp_dose_D1', 'cp_dose_D2','drug_id']] #,'cp_dose','cp_time']]
len(feature_cols_ini)

974

In [26]:
feature_cols_ini_nocps = [c for c in feature_cols if c not in ['kfold','sig_id', 'cp_time_24', 'cp_time_48', 'cp_time_72', 'cp_dose_D1', 'cp_dose_D2','drug_id']]
feature_cols_ini_nocps = [c for c in feature_cols_ini_nocps if 'kmeans_' not in c]
len(feature_cols_ini_nocps)

955

In [27]:
feature_cols_ini

['pca_G-0',
 'pca_G-1',
 'pca_G-2',
 'pca_G-3',
 'pca_G-4',
 'pca_G-5',
 'pca_G-6',
 'pca_G-7',
 'pca_G-8',
 'pca_G-9',
 'pca_G-10',
 'pca_G-11',
 'pca_G-12',
 'pca_G-13',
 'pca_G-14',
 'pca_G-15',
 'pca_G-16',
 'pca_G-17',
 'pca_G-18',
 'pca_G-19',
 'pca_G-20',
 'pca_G-21',
 'pca_G-22',
 'pca_G-23',
 'pca_G-24',
 'pca_G-25',
 'pca_G-26',
 'pca_G-27',
 'pca_G-28',
 'pca_C-0',
 'pca_C-1',
 'pca_C-2',
 'pca_C-3',
 'g-0',
 'g-1',
 'g-2',
 'g-3',
 'g-4',
 'g-6',
 'g-8',
 'g-11',
 'g-12',
 'g-13',
 'g-14',
 'g-16',
 'g-17',
 'g-19',
 'g-20',
 'g-21',
 'g-22',
 'g-24',
 'g-26',
 'g-27',
 'g-28',
 'g-29',
 'g-30',
 'g-32',
 'g-33',
 'g-34',
 'g-35',
 'g-36',
 'g-37',
 'g-38',
 'g-39',
 'g-40',
 'g-41',
 'g-43',
 'g-44',
 'g-45',
 'g-46',
 'g-47',
 'g-48',
 'g-49',
 'g-50',
 'g-51',
 'g-52',
 'g-53',
 'g-55',
 'g-56',
 'g-57',
 'g-58',
 'g-59',
 'g-60',
 'g-61',
 'g-62',
 'g-63',
 'g-64',
 'g-65',
 'g-66',
 'g-67',
 'g-68',
 'g-69',
 'g-70',
 'g-71',
 'g-72',
 'g-73',
 'g-75',
 'g-76',
 'g-77'

In [28]:
# Standarize
for colname in feature_cols_ini:
    valor_fold = folds[colname].values
    valor_tst = test_noctl[colname].values
    mean_v = np.mean(valor_fold)
    std_v = np.std(valor_fold)
    if std_v==0:
        std_v=1e-5
    folds[colname] = (valor_fold-mean_v)/std_v
    test_noctl[colname] = (valor_tst-mean_v)/std_v
    print(colname, mean_v, std_v)

pca_G-0 -3.165934989349514e-05 1.000022942477717
pca_G-1 2.319063523304048e-05 0.9995899673134725
pca_G-2 -5.652831749957289e-05 0.9998962469475887
pca_G-3 0.0001490332009944081 1.0001572798115808
pca_G-4 3.974275730490028e-05 0.9997665499591255
pca_G-5 7.361036404477721e-05 0.9997785695567266
pca_G-6 -3.722530770014762e-05 0.999715481686792
pca_G-7 -4.176801412012035e-05 1.000241038168113
pca_G-8 -3.368941635790669e-05 0.9998038447404771
pca_G-9 5.615480974268078e-05 0.9997265629134643
pca_G-10 -9.040019011924138e-05 0.9996772931738682
pca_G-11 -6.078795307598129e-05 0.9997559892025472
pca_G-12 -5.8613588758447095e-05 0.9994258399589678
pca_G-13 -9.893185187003101e-05 0.9997849369040596
pca_G-14 -6.201536405669977e-05 0.9997168847993001
pca_G-15 -8.278464554968567e-05 0.999498292430778
pca_G-16 -3.8333825188847665e-06 1.0007720562770313
pca_G-17 -1.2060393554003437e-05 0.9995915837385392
pca_G-18 -6.393268281031218e-05 1.0000080139572307
pca_G-19 4.308745677197324e-05 0.99993752822325

### Train

In [29]:
# def run_training(fold, seed, display=2):
#     seed_everything(seed)
#     train = folds.copy()
#     test_ = test_noctl.copy()
    
#     trn_idx = train[train['kfold'] != fold].index
#     val_idx = train[train['kfold'] == fold].index
    
#     train_df = train[train['kfold'] != fold].reset_index(drop=True)
#     valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
#     x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
#     x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
#     train_dataset = MoADataset(x_train, y_train)
#     valid_dataset = MoADataset(x_valid, y_valid)
#     trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#     validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     model = Model(
#         num_features=num_features,
#         num_targets=num_targets,
#         hidden_size=hidden_size,
#     )
#     model.to(DEVICE)
    
#     optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
#     scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
#                                               max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
# #     scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=3)
    
# #     loss_fn = nn.BCEWithLogitsLoss()
#     loss_val = nn.BCEWithLogitsLoss()
#     loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
#     early_stopping_steps = EARLY_STOPPING_STEPS
#     early_step = 0
    
#     oof = np.zeros((len(train), num_targets))
#     best_loss = np.inf
#     best_epoch = 0
#     res = []
#     for epoch in range(EPOCHS):
        
#         train_loss, mloss_train = train_fn(model, optimizer, scheduler, loss_tr, trainloader, DEVICE)
                
# #         print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
#         valid_loss, mloss_valid, y_true, valid_preds = valid_fn(model, loss_val, validloader, DEVICE)
    
#         if mloss_valid < best_loss:
#             best_loss = mloss_valid
#             best_epoch = epoch
#             oof[val_idx] = valid_preds
#             torch.save(model.state_dict(), output_dir / f'ann_model_seed{seed}_fold__{fold}.pth')
            
#         elif(EARLY_STOP == True):
#             early_step += 1
#             if (early_step >= early_stopping_steps):
#                 break
#         if scheduler.__class__ !=  torch.optim.lr_scheduler.ReduceLROnPlateau:
#             lr = scheduler.get_last_lr()
#         else:
#             lr = [0.0]
        
#         # Save results
#         # ------------
#         res.append(dict({'epoch':epoch, 'lr':lr[0], 'trn_loss':mloss_train, 'val_loss':mloss_valid, 'best_epoch':best_epoch, 'best_loss':best_loss}))
        
#         res_df = pd.DataFrame(res)
#         res_df.to_csv(output_dir / f'res_seed{seed}_fold{fold}_.csv')
        
#         fig, ax = plt.subplots(nrows=1, ncols=1 )
#         min_val = res_df[['trn_loss','val_loss']].min().min()
#         ax.plot(res_df['trn_loss'])
#         ax.plot(res_df['val_loss'])
#         plt.ylim((min_val,0.020))
#         plt.title(f"logloss in fold={fold} min={res_df['val_loss'].min()}")
#         fig.savefig(output_dir / f'loss_seed{seed}_fold{fold}_.png')
#         plt.close(fig)
        
#         fig, ax = plt.subplots(nrows=1, ncols=1 )
#         ax.plot(res_df['lr'])
#         plt.title(f"lr in fold={fold}")
#         fig.savefig(output_dir / f'lr_seed{seed}_fold{fold}_.png')
#         plt.close(fig)
        
#         if display==2:
#             print(f"SEED:{seed} FOLD:{fold}, EPOCH:{epoch:2d}, lr:{lr[0]:.9f} trn_loss:{mloss_train:.6f}, val_loss:{mloss_valid:.6f}, best_epoch:{best_epoch}, best_loss:{best_loss:.6f}")

#     if display==1:
#         print(f"SEED:{seed} FOLD:{fold}, EPOCH:{epoch:2d}, lr:{lr[0]:.9f} trn_loss:{mloss_train:.6f}, val_loss:{mloss_valid:.6f}, best_epoch:{best_epoch}, best_loss:{best_loss:.6f}")
        
    
#     #--------------------- PREDICTION---------------------
#     x_test = test_[feature_cols].values
#     testdataset = TestDataset(x_test)
#     testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     model = Model(
#         num_features=num_features,
#         num_targets=num_targets,
#         hidden_size=hidden_size,
#     )
    
#     model.load_state_dict(torch.load(output_dir / f'ann_model_seed{seed}_fold__{fold}.pth'))
#     model.to(DEVICE)
    
#     predictions_tst = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
#     predictions_tst = inference_fn(model, testloader, DEVICE)
    
#     return oof, predictions_tst, best_loss


In [30]:
def run_inference_ANN(fold, seedfold, seedrun, display=2):
    seed_everything(seedrun)
    test_ = test_noctl.copy()
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )

    model.load_state_dict(torch.load(f'{directorio}/ann_model_seedfold{seedfold}_seedrun{seedrun}_fold__{fold}.pth'))
    model.to(DEVICE)
    
    predictions_tst = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions_tst = inference_fn(model, testloader, DEVICE)
    
    return predictions_tst 

In [31]:
def run_k_fold(NFOLDS, seed_fold, seed_run, display=2):
    predictions_kfold = np.zeros((len(test_noctl), len(target_cols)))
    for fold in range(NFOLDS):
        pred_ = run_inference_ANN(fold, seed_fold, seed_run, display)
        predictions_kfold += pred_ / NFOLDS
    return predictions_kfold

In [32]:
len(feature_cols_ini)

974

> ### Train ANN models 29 y 28 de nov

In [33]:
directorio = '../input/annrandom' #'../input/annstratkmean7' #'../input/annrandomnosmooth' #'../input/annstratnosmooth' #
folds_cp = folds.copy()
feature_cols = feature_cols_ini.copy()
NFOLDS = CFG.num_folds
num_features=len(feature_cols)
num_targets=len(target_cols)

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False
hidden_size = 256
res = []
drop1_feat = 0.05
drop2_feat = 0.05
drop3_feat = 0.25


# Averaging on multiple SEEDS
oof_seed = np.zeros((len(folds), len(target_cols)))
predictions = np.zeros((len(test_noctl), len(target_cols)))
losses_list = []
# SEED = [[0,1],[1,1],[2,1],[3,1]]
SEED = np.arange(16)
for seed in tqdm(SEED):
    seed_fold = seed // 4
    seed_run = seed % 4
    predictions_ = run_k_fold(NFOLDS, seed_fold, seed_run, display=2)
    predictions += predictions_

# FINAL CV LOGLOSS
for col in target_cols:
    test_noctl[col] = 0.0
test_noctl[target_cols] = predictions / len(SEED)

In [34]:
submission = sample_submission.drop(columns=target_cols)\
.merge(test_noctl[['sig_id']+target_cols], on='sig_id', how='left')\
.fillna(0.0).reset_index(drop=True)
name_sub = 'submission.csv'
submission.to_csv(name_sub, index=False)
# print(name_sub)
print("ANN_1dic")
submission_ANN_30nov = submission.copy()
submission.head()

ANN_1dic


,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

## ANN del 30 de noviembre

In [35]:
# directorio = '../input/annrandomnosmooth' 
# folds_cp = folds.copy()
# feature_cols = feature_cols_ini.copy()
# NFOLDS = CFG.num_folds
# num_features=len(feature_cols)
# num_targets=len(target_cols)

# DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
# EPOCHS = 25
# BATCH_SIZE = 128
# LEARNING_RATE = 1e-3
# WEIGHT_DECAY = 1e-5
# EARLY_STOPPING_STEPS = 10
# EARLY_STOP = False
# hidden_size = 256
# res = []
# drop1_feat = 0.05
# drop2_feat = 0.05
# drop3_feat = 0.25


# # Averaging on multiple SEEDS
# oof_seed = np.zeros((len(folds), len(target_cols)))
# predictions = np.zeros((len(test_noctl), len(target_cols)))
# losses_list = []
# # SEED = [[0,1],[1,1],[2,1],[3,1]]
# SEED = np.arange(16)
# for seed in tqdm(SEED):
#     seed_fold = seed // 4
#     seed_run = seed % 4
#     predictions_ = run_k_fold(NFOLDS, seed_fold, seed_run, display=2)
#     predictions += predictions_

# # FINAL CV LOGLOSS
# for col in target_cols:
#     test_noctl[col] = 0.0
# test_noctl[target_cols] = predictions / len(SEED)

In [36]:
# submission = sample_submission.drop(columns=target_cols)\
# .merge(test_noctl[['sig_id']+target_cols], on='sig_id', how='left')\
# .fillna(0.0).reset_index(drop=True)
# name_sub = 'submission.csv'
# submission.to_csv(name_sub, index=False)
# print("ANN_30nov")
# submission_ANN_30nov = submission.copy()
# submission.head()

## ANN del 28 de noviembre

In [37]:
# directorio = '../input/annrandom28nov3'
# folds_cp = folds.copy()
# feature_cols = feature_cols_ini_nocps.copy()
# NFOLDS = CFG.num_folds
# num_features=len(feature_cols)
# num_targets=len(target_cols)

# DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
# EPOCHS = 25
# BATCH_SIZE = 128
# LEARNING_RATE = 1e-3
# WEIGHT_DECAY = 1e-5
# EARLY_STOPPING_STEPS = 10
# EARLY_STOP = False
# hidden_size = 256
# res = []
# drop1_feat = 0.05
# drop2_feat = 0.05
# drop3_feat = 0.25


# # Averaging on multiple SEEDS
# oof_seed = np.zeros((len(folds), len(target_cols)))
# predictions = np.zeros((len(test_noctl), len(target_cols)))
# losses_list = []
# # SEED = [[0,1],[1,1],[2,1],[3,1]]
# SEED = np.arange(16)
# for seed in tqdm(SEED):
#     seed_fold = seed // 4
#     seed_run = seed % 4
#     predictions_ = run_k_fold(NFOLDS, seed_fold, seed_run, display=2)
#     predictions += predictions_

# # FINAL CV LOGLOSS
# for col in target_cols:
#     test_noctl[col] = 0.0
# test_noctl[target_cols] = predictions / len(SEED)

In [38]:
# submission = sample_submission.drop(columns=target_cols)\
# .merge(test_noctl[['sig_id']+target_cols], on='sig_id', how='left')\
# .fillna(0.0).reset_index(drop=True)
# # sub.to_csv('submission.csv', index=False)
# # name_sub = 'submission.csv'
# # submission.to_csv(name_sub, index=False)
# print("ANN_28nov")
# submission_ANN_28nov = submission.copy()
# submission.head()

# 2. RESNET

In [39]:
hidden_size1 = 1024
hidden_size2 = hidden_size1 // 2
drop_h1 = 0.2
drop_h2 = 0.2
drop_h3 = 0.2

class Model(nn.Module):
    def __init__(self, num_features, num_targets):
        super(Model, self).__init__()
        # h1
        self.batch_norm1_h1 = nn.BatchNorm1d(num_features)
        self.dropout1_h1 = nn.Dropout(drop_h1) 
        self.dense1_h1 = nn.Linear(num_features, hidden_size1)
        
        self.batch_norm2_h1 = nn.BatchNorm1d(hidden_size1)
        self.dense2_h1 = nn.Linear(hidden_size1, hidden_size2)

        # h2
        self.batch_norm1_h2 = nn.BatchNorm1d(hidden_size2+num_features)
        self.dropout1_h2 = nn.Dropout(drop_h2) 
        self.dense1_h2 = nn.Linear(hidden_size2+num_features, hidden_size1)
        
        self.batch_norm2_h2 = nn.BatchNorm1d(hidden_size1)
        self.dense2_h2 = nn.Linear(hidden_size1,hidden_size1)
        
        self.batch_norm3_h2 = nn.BatchNorm1d(hidden_size1)
        self.dense3_h2 = nn.Linear(hidden_size1,hidden_size2)
               
        self.batch_norm4_h2 = nn.BatchNorm1d(hidden_size2)
        self.dense4_h2 = nn.Linear(hidden_size2,hidden_size2)
        
        # h3
#         self.batch_norm1_h3 = nn.BatchNorm1d(hidden_size2*2)
#         self.dropout1_h3 = nn.Dropout(drop_h3) 
#         self.dense1_h3 = nn.Linear(hidden_size2*2,hidden_size2)

        self.batch_norm1_h3 = nn.BatchNorm1d(hidden_size2)
        self.dropout1_h3 = nn.Dropout(drop_h3) 
        self.dense1_h3 = nn.Linear(hidden_size2,hidden_size2//2)
        
        self.batch_norm2_h3 = nn.BatchNorm1d(hidden_size2//2)
        self.dropout2_h3 = nn.Dropout(drop_h3) 
        self.dense2_h3 = nn.Linear(hidden_size2//2,num_targets)
        
#         self.batch_norm3_h3 = nn.BatchNorm1d(num_targets)
#         self.dense3_h3 = nn.Linear(num_targets,num_targets)
        
#     x2 = torch.log(1+x-torch.min(x))
#         x2 = self.batch_norm1B(x2)
#         x2 = self.dropout1B(x2)
#         x2 = self.dense1B(x2)
#         x2 = torch.exp(x2)
#         x2 = F.leaky_relu(x2)
        
        
    def forward(self, x):
        xmult = x-torch.min(x)
        
        h1 = self.batch_norm1_h1(x)
        h1 = self.dropout1_h1(h1)
#         h1 = F.elu(self.dense1_h1(h1))
        h1 = F.leaky_relu(self.dense1_h1(h1))
        h1 = self.batch_norm2_h1(h1)
#         h1 = F.elu(self.dense2_h1(h1))
        h1 = F.leaky_relu(self.dense2_h1(h1))
    
        combined = torch.cat((x.view(x.size(0), -1), h1.view(h1.size(0), -1)), dim=1)
        h2 = self.batch_norm1_h2(combined)
        h2 = self.dropout1_h2(h2)
#         h2 = F.relu(self.dense1_h2(h2))
        h2 = F.leaky_relu(self.dense1_h2(h2))
        h2 = self.batch_norm2_h2(h2)
#         h2 = F.elu(self.dense2_h2(h2))
        h2 = F.leaky_relu(self.dense2_h2(h2))
        h2 = self.batch_norm3_h2(h2)
#         h2 = F.relu(self.dense3_h2(h2))
        h2 = F.leaky_relu(self.dense3_h2(h2))
        h2 = self.batch_norm4_h2(h2)
#         h2 = F.elu(self.dense4_h2(h2))
        h2 = F.leaky_relu(self.dense4_h2(h2))
    
#         # Xmult
#         xmult = x-torch.min(x)
#         h1b = torch.log1p(xmult)
#         h1b = self.batch_norm1_h1(h1b)
#         h1b = self.dropout1_h1(h1b)
#         h1b = self.dense1_h1(h1b)
#         h1b = torch.expm1(h1b)
#         h1b = F.leaky_relu(h1b)
        
#         h1b = self.batch_norm2_h1(h1b)
# #         h1 = F.elu(self.dense2_h1(h1))
#         h1b = F.leaky_relu(self.dense2_h1(h1b))
    
#         combined_mult = torch.cat((x.view(x.size(0), -1), h1b.view(h1b.size(0), -1)), dim=1)
#         h2b = combined_mult-torch.min(combined_mult)
#         h2b = torch.log1p(h2b)
#         h2b = self.batch_norm1_h2(h2b)
#         h2b = self.dropout1_h2(h2b)
#         h2b = self.dense1_h2(h2b)
#         h2b = torch.expm1(h2b)
#         h2b = F.leaky_relu(h2b)

#         h2b = self.batch_norm2_h2(h2b)
# #         h2 = F.elu(self.dense2_h2(h2))
#         h2b = F.leaky_relu(self.dense2_h2(h2b))
#         h2b = self.batch_norm3_h2(h2b)
# #         h2 = F.relu(self.dense3_h2(h2))
#         h2b = F.leaky_relu(self.dense3_h2(h2b))
#         h2b = self.batch_norm4_h2(h2b)
# #         h2 = F.elu(self.dense4_h2(h2))
#         h2b = F.leaky_relu(self.dense4_h2(h2b))
    
    
#         combined_h1s = torch.cat((h1.view(h1.size(0), -1), h1b.view(h1b.size(0), -1)), dim=1)
#         combined_h2s = torch.cat((h2.view(h2.size(0), -1), h2b.view(h2b.size(0), -1)), dim=1)
    
#         average2heads= (combined_h1s+combined_h2s)/2.0
        average2heads = (h1+h2)/2.0
        h3 = self.batch_norm1_h3(average2heads)
#         h3 = self.dropout1_h3(h3)
        h3 = self.dense1_h3(h3)
        
        output = self.batch_norm2_h3(h3)
#         output = self.dropout2_h3(output)
        output = self.dense2_h3(output)
        return output

In [40]:
def run_inference_RESNET(fold, seedfold, seedrun, display=2):
    seed_everything(seedrun)
    test_ = test_noctl.copy()
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
#         hidden_size=hidden_size,
    )

    model.load_state_dict(torch.load(f'{directorio}/ann_model_seedfold{seedfold}_seedrun{seedrun}_fold__{fold}.pth'))
    model.to(DEVICE)
    
    predictions_tst = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions_tst = inference_fn(model, testloader, DEVICE)
    
    return predictions_tst

In [41]:
def run_k_fold(NFOLDS, seed_fold, seed_run, display=2):
    predictions_kfold = np.zeros((len(test_noctl), len(target_cols)))
    for fold in range(NFOLDS):
        pred_ = run_inference_RESNET(fold, seed_fold, seed_run, display)
        predictions_kfold += pred_ / NFOLDS
    return predictions_kfold

## Resnet del 30 de noviembre

In [42]:
directorio = '../input/resnetrandom' #'../input/resnetstrat' #'../input/resnetrandomnostrat' #'../input/resnetstratnosmooth' #'../input/resnetrandomnostrat'
folds_cp = folds.copy()
feature_cols = feature_cols_ini.copy()
NFOLDS = CFG.num_folds
num_features=len(feature_cols)
num_targets=len(target_cols)

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

res = []
# Averaging on multiple SEEDS
oof_seed = np.zeros((len(folds), len(target_cols)))
predictions = np.zeros((len(test_noctl), len(target_cols)))
losses_list = []
# SEED = [[0,3],[1,3],[2,2],[3,0]]
# for seed_fold, seed_run in tqdm(SEED):
SEED = np.arange(16)
for seed in tqdm(SEED):
    seed_fold = seed // 4
    seed_run = seed % 4
    predictions_ = run_k_fold(NFOLDS, seed_fold, seed_run, display=2)
    predictions += predictions_

# FINAL CV LOGLOSS
for col in target_cols:
    test_noctl[col] = 0.0
test_noctl[target_cols] = predictions / len(SEED)

In [43]:
submission = sample_submission.drop(columns=target_cols)\
.merge(test_noctl[['sig_id']+target_cols], on='sig_id', how='left')\
.fillna(0.0).reset_index(drop=True)
name_sub = 'submission.csv'
submission.to_csv(name_sub, index=False)
print("RESNET 30nov")
submission_RESNET_30nov = submission.copy()
submission.head()

RESNET 30nov


,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

## Resnet del 28 de noviembre

In [44]:
# directorio = '../input/resnetrandom28nov'
# folds_cp = folds.copy()
# feature_cols = feature_cols_ini_nocps.copy()
# NFOLDS = CFG.num_folds
# num_features=len(feature_cols)
# num_targets=len(target_cols)

# DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
# EPOCHS = 25
# BATCH_SIZE = 128
# LEARNING_RATE = 1e-3
# WEIGHT_DECAY = 1e-5
# EARLY_STOPPING_STEPS = 10
# EARLY_STOP = False

# res = []
# # Averaging on multiple SEEDS
# oof_seed = np.zeros((len(folds), len(target_cols)))
# predictions = np.zeros((len(test_noctl), len(target_cols)))
# losses_list = []
# # SEED = [[0,3],[1,3],[2,2],[3,0]]
# # for seed_fold, seed_run in tqdm(SEED):
# SEED = np.arange(16)
# for seed in tqdm(SEED):
#     seed_fold = seed // 4
#     seed_run = seed % 4
#     predictions_ = run_k_fold(NFOLDS, seed_fold, seed_run, display=2)
#     predictions += predictions_

# # FINAL CV LOGLOSS
# for col in target_cols:
#     test_noctl[col] = 0.0
# test_noctl[target_cols] = predictions / len(SEED)

In [45]:
# submission = sample_submission.drop(columns=target_cols)\
# .merge(test_noctl[['sig_id']+target_cols], on='sig_id', how='left')\
# .fillna(0.0).reset_index(drop=True)
# # sub.to_csv('submission.csv', index=False)
# # name_sub = 'submission.csv'
# # submission.to_csv(name_sub, index=False)
# # print(name_sub)
# print("RESNET 28nov")
# submission_RESNET_28nov = submission.copy()
# submission.head()

# 3. TABNET

In [46]:
sys.path.insert(0, "../input/tabnetdevelop/tabnet-develop")
seed_everything(seed=42)

In [47]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

def process_data_tabnet(data):
    data.loc[:, 'cp_time'] = data.loc[:, 'cp_time'].map({24: 0, 48: 1, 72: 2})
    data.loc[:, 'cp_dose'] = data.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    return data    

In [48]:
# folds = process_data_tabnet(folds_novar_enc)
# test_noctl = process_data_tabnet(test_noctl_novar_enc)
# print(folds.shape, test_noctl.shape)

# del folds_novar_enc
# del test_noctl_novar_enc
# gc.collect()

# feature_cols = [c for c in folds.columns if c not in target_cols]
# if not CFG.original_feats: feature_cols = [c for c in feature_cols if c not in GENES+CELLS]
# feature_cols_ini = [c for c in feature_cols if c not in ['kfold','sig_id','cp_time','cp_dose']] #, 'cp_time_24', 'cp_time_48', 'cp_time_72', 'cp_dose_D1', 'cp_dose_D2']] #,'cp_dose','cp_time']]
# len(feature_cols_ini)
# categorical_feats = ['cp_time','cp_dose']
# numeric_feats = [c for c in feature_cols_ini if c not in ['cp_time','cp_dose']]

In [49]:
# # Standarize Numerical Features
# for colname in numeric_feats:
#     valor_fold = folds[colname].values
#     valor_tst = test_noctl[colname].values
#     mean_v = np.mean(valor_fold)
#     std_v = np.std(valor_fold)
#     if std_v==0:
#         std_v=1e-5
#     folds[colname] = (valor_fold-mean_v)/std_v
#     test_noctl[colname] = (valor_tst-mean_v)/std_v
#     print(colname, mean_v, std_v)

In [50]:
## TABNET https://www.kaggle.com/ludovick/introduction-to-tabnet-kfold-10-training
#from pytorch_tabnet.tab_model import TabModel
from scipy.sparse import csc_matrix
import time
from abc import abstractmethod
from pytorch_tabnet import tab_network
from pytorch_tabnet.multiclass_utils import unique_labels
from sklearn.metrics import roc_auc_score, mean_squared_error, accuracy_score
from torch.nn.utils import clip_grad_norm_
from pytorch_tabnet.utils import (PredictDataset,
                                  create_dataloaders,
                                  create_explain_matrix)
from sklearn.base import BaseEstimator
from torch.utils.data import DataLoader
from copy import deepcopy
import io
import json
import shutil
import zipfile

In [51]:
# def evals(model, X, y, verbose=True):
#     with torch.no_grad():
#         y_preds = model.predict(X)
#         y_preds = torch.clamp(y_preds, 0.0, 1.0).detach().numpy()
#     score = log_loss_multi(y, y_preds)
#     #print("Logloss = ", score)
#     return y_preds, score


# def inference_fn(model, X ,verbose=True):
#     with torch.no_grad():
#         y_preds = model.predict( X )
#         y_preds = torch.sigmoid(torch.as_tensor(y_preds)).numpy()
#     return y_preds


# def log_loss_score(actual, predicted,  eps=1e-15):

#         """
#         :param predicted:   The predicted probabilities as floats between 0-1
#         :param actual:      The binary labels. Either 0 or 1.
#         :param eps:         Log(0) is equal to infinity, so we need to offset our predicted values slightly by eps from 0 or 1
#         :return:            The logarithmic loss between between the predicted probability assigned to the possible outcomes for item i, and the actual outcome.
#         """

        
#         p1 = actual * np.log(predicted+eps)
#         p0 = (1-actual) * np.log(1-predicted+eps)
#         loss = p0 + p1

#         return -loss.mean()
    
# def log_loss_multi(y_true, y_pred):
#     M = y_true.shape[1]
#     results = np.zeros(M)
#     for i in range(M):
#         results[i] = log_loss_score(y_true[:,i], y_pred[:,i])
#     return results.mean()

def check_targets(targets):
    ### check if targets are all binary in training set
    
    for i in range(targets.shape[1]):
        if len(np.unique(targets[:,i])) != 2:
            return False
    return True
            

In [52]:
class TabModel(BaseEstimator):
    def __init__(self, n_d=8, n_a=8, n_steps=3, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=1,
                 n_independent=2, n_shared=2, epsilon=1e-15,  momentum=0.02,
                 lambda_sparse=1e-3, seed=0,
                 clip_value=1, verbose=1,
                 optimizer_fn=torch.optim.Adam,
                 optimizer_params=dict(lr=2e-2),
                 scheduler_params=None, scheduler_fn=None,
                 mask_type="sparsemax",
                 input_dim=None, output_dim=None,
                 device_name='auto'):
        """ Class for TabNet model
        Parameters
        ----------
            device_name: str
                'cuda' if running on GPU, 'cpu' if not, 'auto' to autodetect
        """

        self.n_d = n_d
        self.n_a = n_a
        self.n_steps = n_steps
        self.gamma = gamma
        self.cat_idxs = cat_idxs
        self.cat_dims = cat_dims
        self.cat_emb_dim = cat_emb_dim
        self.n_independent = n_independent
        self.n_shared = n_shared
        self.epsilon = epsilon
        self.momentum = momentum
        self.lambda_sparse = lambda_sparse
        self.clip_value = clip_value
        self.verbose = verbose
        self.optimizer_fn = optimizer_fn
        self.optimizer_params = optimizer_params
        self.device_name = device_name
        self.scheduler_params = scheduler_params
        self.scheduler_fn = scheduler_fn
        self.mask_type = mask_type
        self.input_dim = input_dim
        self.output_dim = output_dim

        self.batch_size = 1024

        self.seed = seed
        torch.manual_seed(self.seed)
        # Defining device
        if device_name == 'auto':
            if torch.cuda.is_available():
                device_name = 'cuda'
            else:
                device_name = 'cpu'
        self.device = torch.device(device_name)
        print(f"Device used : {self.device}")

    @abstractmethod
    def construct_loaders(self, X_train, y_train, X_valid, y_valid,
                          weights, batch_size, num_workers, drop_last):
        """
        Returns
        -------
        train_dataloader, valid_dataloader : torch.DataLoader, torch.DataLoader
            Training and validation dataloaders
        -------
        """
        raise NotImplementedError('users must define construct_loaders to use this base class')

    def init_network(
                     self,
                     input_dim,
                     output_dim,
                     n_d,
                     n_a,
                     n_steps,
                     gamma,
                     cat_idxs,
                     cat_dims,
                     cat_emb_dim,
                     n_independent,
                     n_shared,
                     epsilon,
                     virtual_batch_size,
                     momentum,
                     device_name,
                     mask_type,
                     ):
        self.network = tab_network.TabNet(
            input_dim,
            output_dim,
            n_d=n_d,
            n_a=n_a,
            n_steps=n_steps,
            gamma=gamma,
            cat_idxs=cat_idxs,
            cat_dims=cat_dims,
            cat_emb_dim=cat_emb_dim,
            n_independent=n_independent,
            n_shared=n_shared,
            epsilon=epsilon,
            virtual_batch_size=virtual_batch_size,
            momentum=momentum,
            device_name=device_name,
            mask_type=mask_type).to(self.device)

        self.reducing_matrix = create_explain_matrix(
            self.network.input_dim,
            self.network.cat_emb_dim,
            self.network.cat_idxs,
            self.network.post_embed_dim)

    def fit(self, X_train, y_train, X_valid=None, y_valid=None, loss_fn=None,
            weights=0, max_epochs=100, patience=10, batch_size=1024,
            virtual_batch_size=128, num_workers=0, drop_last=False):
        """Train a neural network stored in self.network
        Using train_dataloader for training data and
        valid_dataloader for validation.
        Parameters
        ----------
            X_train: np.ndarray
                Train set
            y_train : np.array
                Train targets
            X_train: np.ndarray
                Train set
            y_train : np.array
                Train targets
            weights : bool or dictionnary
                0 for no balancing
                1 for automated balancing
                dict for custom weights per class
            max_epochs : int
                Maximum number of epochs during training
            patience : int
                Number of consecutive non improving epoch before early stopping
            batch_size : int
                Training batch size
            virtual_batch_size : int
                Batch size for Ghost Batch Normalization (virtual_batch_size < batch_size)
            num_workers : int
                Number of workers used in torch.utils.data.DataLoader
            drop_last : bool
                Whether to drop last batch during training
        """
        # update model name

        self.update_fit_params(X_train, y_train, X_valid, y_valid, loss_fn,
                               weights, max_epochs, patience, batch_size,
                               virtual_batch_size, num_workers, drop_last)

        train_dataloader, valid_dataloader = self.construct_loaders(X_train,
                                                                    y_train,
                                                                    X_valid,
                                                                    y_valid,
                                                                    self.updated_weights,
                                                                    self.batch_size,
                                                                    self.num_workers,
                                                                    self.drop_last)

        self.init_network(
            input_dim=self.input_dim,
            output_dim=self.output_dim,
            n_d=self.n_d,
            n_a=self.n_a,
            n_steps=self.n_steps,
            gamma=self.gamma,
            cat_idxs=self.cat_idxs,
            cat_dims=self.cat_dims,
            cat_emb_dim=self.cat_emb_dim,
            n_independent=self.n_independent,
            n_shared=self.n_shared,
            epsilon=self.epsilon,
            virtual_batch_size=self.virtual_batch_size,
            momentum=self.momentum,
            device_name=self.device_name,
            mask_type=self.mask_type
        )

        self.optimizer = self.optimizer_fn(self.network.parameters(),
                                           **self.optimizer_params)

        if self.scheduler_fn:
            self.scheduler = self.scheduler_fn(self.optimizer, **self.scheduler_params)
        else:
            self.scheduler = None

        self.losses_train = []
        self.losses_valid = []
        self.learning_rates = []
        self.metrics_train = []
        self.metrics_valid = []

        if self.verbose > 0:
            print("Will train until validation stopping metric",
                  f"hasn't improved in {self.patience} rounds.")
            msg_epoch = f'| EPOCH |  train  |   valid  | total time (s)'
            print('---------------------------------------')
            print(msg_epoch)

        total_time = 0
        while (self.epoch < self.max_epochs and
               self.patience_counter < self.patience):
            starting_time = time.time()
            # updates learning rate history
            self.learning_rates.append(self.optimizer.param_groups[-1]["lr"])

            fit_metrics = self.fit_epoch(train_dataloader, valid_dataloader)

            # leaving it here, may be used for callbacks later
            self.losses_train.append(fit_metrics['train']['loss_avg'])
            self.losses_valid.append(fit_metrics['valid']['total_loss'])
            self.metrics_train.append(fit_metrics['train']['stopping_loss'])
            self.metrics_valid.append(fit_metrics['valid']['stopping_loss'])

            stopping_loss = fit_metrics['valid']['stopping_loss']
            if stopping_loss < self.best_cost:
                self.best_cost = stopping_loss
                self.patience_counter = 0
                # Saving model
                self.best_network = deepcopy(self.network)
                has_improved = True
            else:
                self.patience_counter += 1
                has_improved=False
            self.epoch += 1
            total_time += time.time() - starting_time
            if self.verbose > 0:
                if self.epoch % self.verbose == 0:
                    separator = "|"
                    msg_epoch = f"| {self.epoch:<5} | "
                    msg_epoch += f" {fit_metrics['train']['stopping_loss']:.5f}"
                    msg_epoch += f' {separator:<2} '
                    msg_epoch += f" {fit_metrics['valid']['stopping_loss']:.5f}"
                    msg_epoch += f' {separator:<2} '
                    msg_epoch += f" {np.round(total_time, 1):<10}"
                    msg_epoch += f" {has_improved}"
                    print(msg_epoch)

        if self.verbose > 0:
            if self.patience_counter == self.patience:
                print(f"Early stopping occured at epoch {self.epoch}")
            print(f"Training done in {total_time:.3f} seconds.")
            print('---------------------------------------')

        self.history = {"train": {"loss": self.losses_train,
                                  "metric": self.metrics_train,
                                  "lr": self.learning_rates},
                        "valid": {"loss": self.losses_valid,
                                  "metric": self.metrics_valid}}
        # load best models post training
        self.load_best_model()

        # compute feature importance once the best model is defined
        self._compute_feature_importances(train_dataloader)

    def save_model(self, path):
        """
        Saving model with two distinct files.
        """
        saved_params = {}
        for key, val in self.get_params().items():
            if isinstance(val, type):
                # Don't save torch specific params
                continue
            else:
                saved_params[key] = val

        # Create folder
        Path(path).mkdir(parents=True, exist_ok=True)

        # Save models params
        with open(Path(path).joinpath("model_params.json"), "w", encoding="utf8") as f:
            json.dump(saved_params, f)

        # Save state_dict
        torch.save(self.network.state_dict(), Path(path).joinpath("network.pt"))
        shutil.make_archive(path, 'zip', path)
        shutil.rmtree(path)
        print(f"Successfully saved model at {path}.zip")
        return f"{path}.zip"

    def load_model(self, filepath):

        try:
            with open(filepath + '/model_params.json') as f:
                loaded_params = json.load(f)
            saved_state_dict = torch.load(filepath + '/network.pt')
            
#             with zipfile.ZipFile(filepath) as z:
#                 with z.open("model_params.json") as f:
#                     loaded_params = json.load(f)
#                 with z.open("network.pt") as f:
#                     try:
#                         saved_state_dict = torch.load(f)
#                     except io.UnsupportedOperation:
#                         # In Python <3.7, the returned file object is not seekable (which at least
#                         # some versions of PyTorch require) - so we'll try buffering it in to a
#                         # BytesIO instead:
#                         saved_state_dict = torch.load(io.BytesIO(f.read()))
        except KeyError:
            raise KeyError("Your zip file is missing at least one component")

        self.__init__(**loaded_params)

        self.init_network(
            input_dim=self.input_dim,
            output_dim=self.output_dim,
            n_d=self.n_d,
            n_a=self.n_a,
            n_steps=self.n_steps,
            gamma=self.gamma,
            cat_idxs=self.cat_idxs,
            cat_dims=self.cat_dims,
            cat_emb_dim=self.cat_emb_dim,
            n_independent=self.n_independent,
            n_shared=self.n_shared,
            epsilon=self.epsilon,
            virtual_batch_size=1024,
            momentum=self.momentum,
            device_name=self.device_name,
            mask_type=self.mask_type
        )
        self.network.load_state_dict(saved_state_dict)
        self.network.eval()
        return

    def fit_epoch(self, train_dataloader, valid_dataloader):
        """
        Evaluates and updates network for one epoch.
        Parameters
        ----------
            train_dataloader: a :class: `torch.utils.data.Dataloader`
                DataLoader with train set
            valid_dataloader: a :class: `torch.utils.data.Dataloader`
                DataLoader with valid set
        """
        train_metrics = self.train_epoch(train_dataloader)
        valid_metrics = self.predict_epoch(valid_dataloader)

        fit_metrics = {'train': train_metrics,
                       'valid': valid_metrics}

        return fit_metrics

    @abstractmethod
    def train_epoch(self, train_loader):
        """
        Trains one epoch of the network in self.network
        Parameters
        ----------
            train_loader: a :class: `torch.utils.data.Dataloader`
                DataLoader with train set
        """
        raise NotImplementedError('users must define train_epoch to use this base class')

    @abstractmethod
    def train_batch(self, data, targets):
        """
        Trains one batch of data
        Parameters
        ----------
            data: a :tensor: `torch.tensor`
                Input data
            target: a :tensor: `torch.tensor`
                Target data
        """
        raise NotImplementedError('users must define train_batch to use this base class')

    @abstractmethod
    def predict_epoch(self, loader):
        """
        Validates one epoch of the network in self.network
        Parameters
        ----------
            loader: a :class: `torch.utils.data.Dataloader`
                    DataLoader with validation set
        """
        raise NotImplementedError('users must define predict_epoch to use this base class')

    @abstractmethod
    def predict_batch(self, data, targets):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            batch_outs: dict
        """
        raise NotImplementedError('users must define predict_batch to use this base class')

    def load_best_model(self):
        if self.best_network is not None:
            self.network = self.best_network

    @abstractmethod
    def predict(self, X):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            predictions: np.array
                Predictions of the regression problem or the last class
        """
        raise NotImplementedError('users must define predict to use this base class')

    def explain(self, X):
        """
        Return local explanation
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            M_explain: matrix
                Importance per sample, per columns.
            masks: matrix
                Sparse matrix showing attention masks used by network.
        """
        self.network.eval()

        dataloader = DataLoader(PredictDataset(X),
                                batch_size=self.batch_size, shuffle=False)

        for batch_nb, data in enumerate(dataloader):
            data = data.to(self.device).float()

            M_explain, masks = self.network.forward_masks(data)
            for key, value in masks.items():
                masks[key] = csc_matrix.dot(value.cpu().detach().numpy(),
                                            self.reducing_matrix)

            if batch_nb == 0:
                res_explain = csc_matrix.dot(M_explain.cpu().detach().numpy(),
                                             self.reducing_matrix)
                res_masks = masks
            else:
                res_explain = np.vstack([res_explain,
                                         csc_matrix.dot(M_explain.cpu().detach().numpy(),
                                                        self.reducing_matrix)])
                for key, value in masks.items():
                    res_masks[key] = np.vstack([res_masks[key], value])
        return res_explain, res_masks

    def _compute_feature_importances(self, loader):
        self.network.eval()
        feature_importances_ = np.zeros((self.network.post_embed_dim))
        for data, targets in loader:
            data = data.to(self.device).float()
            M_explain, masks = self.network.forward_masks(data)
            feature_importances_ += M_explain.sum(dim=0).cpu().detach().numpy()

        feature_importances_ = csc_matrix.dot(feature_importances_,
                                              self.reducing_matrix)
        self.feature_importances_ = feature_importances_ / np.sum(feature_importances_)
        
        
class TabNetRegressor(TabModel):

    def construct_loaders(self, X_train, y_train, X_valid, y_valid, weights,
                          batch_size, num_workers, drop_last):
        """
        Returns
        -------
        train_dataloader, valid_dataloader : torch.DataLoader, torch.DataLoader
            Training and validation dataloaders
        -------
        """
        if isinstance(weights, int):
            if weights == 1:
                raise ValueError("Please provide a list of weights for regression.")
        if isinstance(weights, dict):
            raise ValueError("Please provide a list of weights for regression.")

        train_dataloader, valid_dataloader = create_dataloaders(X_train,
                                                                y_train,
                                                                X_valid,
                                                                y_valid,
                                                                weights,
                                                                batch_size,
                                                                num_workers,
                                                                drop_last)
        return train_dataloader, valid_dataloader

    def update_fit_params(self, X_train, y_train, X_valid, y_valid, loss_fn,
                          weights, max_epochs, patience,
                          batch_size, virtual_batch_size, num_workers, drop_last):

        if loss_fn is None:
            self.loss_fn = torch.nn.functional.mse_loss
        else:
            self.loss_fn = loss_fn

        assert X_train.shape[1] == X_valid.shape[1], "Dimension mismatch X_train X_valid"
        self.input_dim = X_train.shape[1]

        if len(y_train.shape) == 1:
            raise ValueError("""Please apply reshape(-1, 1) to your targets
                                if doing single regression.""")
        assert y_train.shape[1] == y_valid.shape[1], "Dimension mismatch y_train y_valid"
        self.output_dim = y_train.shape[1]

        self.updated_weights = weights

        self.max_epochs = max_epochs
        self.patience = patience
        self.batch_size = batch_size
        self.virtual_batch_size = virtual_batch_size
        # Initialize counters and histories.
        self.patience_counter = 0
        self.epoch = 0
        self.best_cost = np.inf
        self.num_workers = num_workers
        self.drop_last = drop_last

    def train_epoch(self, train_loader):
        """
        Trains one epoch of the network in self.network
        Parameters
        ----------
            train_loader: a :class: `torch.utils.data.Dataloader`
                DataLoader with train set
        """

        self.network.train()
        total_loss = 0
        y_pred = []
        y_true = []
        for data, targets in train_loader:
            batch_outs = self.train_batch(data, targets)
            y_pred.append(batch_outs["y_preds"].sigmoid().cpu().detach().numpy())
            y_true.append(batch_outs["y"].cpu().detach().numpy())
            total_loss += batch_outs["loss"]

        y_pred = np.concatenate(y_pred)
        y_true = np.concatenate(y_true)

        #stopping_loss = mean_squared_error(y_true=ys, y_pred=y_preds)
        stopping_loss = log_loss_multi(y_true, y_pred)
        total_loss = total_loss / len(train_loader)
        epoch_metrics = {'loss_avg': total_loss,
                         'stopping_loss': total_loss,
                         }

        if self.scheduler is not None:
            self.scheduler.step()
        return epoch_metrics

    def train_batch(self, data, targets):
        """
        Trains one batch of data
        Parameters
        ----------
            data: a :tensor: `torch.tensor`
                Input data
            target: a :tensor: `torch.tensor`
                Target data
        """
        self.network.train()
        data = data.to(self.device).float()

        targets = targets.to(self.device).float()
        self.optimizer.zero_grad()

        output, M_loss = self.network(data)

        loss = loss_tr(output, targets) #self.loss_fn(output, targets)
        
        loss -= self.lambda_sparse*M_loss

        loss.backward()
        if self.clip_value:
            clip_grad_norm_(self.network.parameters(), self.clip_value)
        self.optimizer.step()

        loss_value = loss.item()
        batch_outs = {'loss': loss_value,
                      'y_preds': output,
                      'y': targets}
        return batch_outs

    def predict_epoch(self, loader):
        """
        Validates one epoch of the network in self.network
        Parameters
        ----------
            loader: a :class: `torch.utils.data.Dataloader`
                    DataLoader with validation set
        """
        
        self.network.eval()
        total_loss = 0

        y_pred = []
        y_true = []
        for data, targets in loader:
            batch_outs = self.predict_batch(data, targets)
            total_loss += batch_outs["loss"]
            y_pred.append(batch_outs["y_preds"].sigmoid().cpu().detach().numpy())
            y_true.append(batch_outs["y"].cpu().detach().numpy())

        y_pred = np.concatenate(y_pred)
        y_true = np.concatenate(y_true)

        stopping_loss = log_loss_multi(y_true, y_pred)

        total_loss = total_loss / len(loader)
        epoch_metrics = {'total_loss': total_loss,
                         'stopping_loss': stopping_loss}

        return epoch_metrics

    def predict_batch(self, data, targets):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            batch_outs: dict
        """
        self.network.eval()
        data = data.to(self.device).float()
        targets = targets.to(self.device).float()

        output, M_loss = self.network(data)
        
        loss = loss_val(output, targets) #self.loss_fn(output, targets)
        #print(self.loss_fn, loss)
        loss -= self.lambda_sparse*M_loss
        #print(loss)
        loss_value = loss.item()
        batch_outs = {'loss': loss_value,
                      'y_preds': output,
                      'y': targets}
        return batch_outs

    def predict(self, X):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            predictions: np.array
                Predictions of the regression problem
        """
        self.network.eval()
        dataloader = DataLoader(PredictDataset(X),
                                batch_size=self.batch_size, shuffle=False)

        y_pred = []
        for batch_nb, data in enumerate(dataloader):
            data = data.to(self.device).float()

            output, M_loss = self.network(data)
            predictions = output.sigmoid().cpu().detach().numpy()
            y_pred.append(predictions)
        y_pred = np.concatenate(y_pred)
        return y_pred

In [53]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [54]:
loss_tr = SmoothBCEwLogits(smoothing =0.001)
loss_val = torch.nn.functional.binary_cross_entropy_with_logits

In [55]:
def run_inference_TABNET(fold, seed_fold, seed_run, display=2):
    seed_everything(seed_run)
    test_ = test_noctl.copy()

#     where_categorical = [i for i, feat in enumerate(feature_cols) if feat in categorical_feats]
    model = TabNetRegressor(n_d=tabnet_parameters['n_d'], n_a=tabnet_parameters['n_a'], 
                        n_steps=tabnet_parameters['n_steps'], gamma=tabnet_parameters['gamma'], 
                        lambda_sparse=tabnet_parameters['lambda_sparse'],mask_type=tabnet_parameters['mask_type'],
#                         cat_dims=[3,2], cat_emb_dim=[1,1], cat_idxs=where_categorical, 
                        optimizer_fn=torch.optim.Adam,
                        optimizer_params=dict(lr=tabnet_parameters['lr'], weight_decay=tabnet_parameters['weight_decay']),
                        device_name='cuda', scheduler_params=dict(milestones=[ 100,150], gamma=0.9), 
                        scheduler_fn=torch.optim.lr_scheduler.MultiStepLR,
                        seed=int(seed_run), verbose=1)
    model.load_model(f'{directorio}/tabnet_model_seedfold{seed_fold}_seedrun{seed_run}_fold__{fold}')
    #--------------------- TEST PREDICTION---------------------
    x_test = test_[feature_cols].values
    x_test = torch.as_tensor(x_test)
    predictions_tst = model.predict(x_test)
    return predictions_tst

In [56]:
def run_k_fold(NFOLDS, seed_fold, seed_run, display=2):
    predictions_kfold = np.zeros((len(test_noctl), len(target_cols)))
    for fold in range(NFOLDS):
        pred_ = run_inference_TABNET(fold, seed_fold, seed_run, display)
        predictions_kfold += pred_ / NFOLDS
    return predictions_kfold

In [57]:
len(feature_cols_ini)

974

## TABNET del 30 de noviembre

In [58]:
directorio = '../input/tabnetrandommean7' #'../input/tabnetstratkmean7' #'../input/tabnetrandomnosmooth' #'../input/tabnetstratnosmooth' #'../input/tabnetrandomnosmooth' #'../input/tabnetstratnosmooth' #'../input/tabnetrandommean7'
tabnet_parameters = {'n_d':35, 'n_a':30, 'n_steps':1, 'gamma':1.3, 'lambda_sparse':0, 'mask_type':'entmax',
                    'max_epochs':200, 'batch_size':1024, 'lr':2e-2, 'weight_decay':1e-5, 'patience':50}


feature_cols = feature_cols_ini.copy()

folds_cp = folds.copy()
feature_cols = feature_cols_ini.copy()
NFOLDS = CFG.num_folds
num_features=len(feature_cols)
num_targets=len(target_cols)

# Averaging on multiple SEEDS
SEED = np.arange(16)
oof_seed = np.zeros((len(folds), len(target_cols)))
predictions = np.zeros((len(test_noctl), len(target_cols)))
losses_list = []
# SEED = [[0,3],[1,1],[2,1],[3,2]]
# for seed_fold, seed_run in tqdm(SEED):
SEED = np.arange(16)
for seed in tqdm(SEED):
    seed_fold = seed // 4
    seed_run = seed % 4

    predictions_ = run_k_fold(NFOLDS, seed_fold, seed_run, display=2)
    predictions += predictions_

# FINAL CV LOGLOSS
for col in target_cols:
    test_noctl[col] = 0.0
test_noctl[target_cols] = predictions / len(SEED)

Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used 

In [59]:
submission = sample_submission.drop(columns=target_cols)\
.merge(test_noctl[['sig_id']+target_cols], on='sig_id', how='left')\
.fillna(0.0).reset_index(drop=True)
name_sub = 'submission.csv'
submission.to_csv(name_sub, index=False)
print('TABNET 30nov')
submission_TABNET_30nov = submission.copy()
submission.head()

TABNET 30nov


,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

## TABNET del 28 de noviembre

In [60]:
# directorio = '../input/tabnetrandom28nov2'
# tabnet_parameters = {'n_d':35, 'n_a':30, 'n_steps':1, 'gamma':1.3, 'lambda_sparse':0, 'mask_type':'entmax',
#                     'max_epochs':200, 'batch_size':1024, 'lr':2e-2, 'weight_decay':1e-5, 'patience':50}

# folds_cp = folds.copy()
# feature_cols = feature_cols_ini_nocps.copy()
# NFOLDS = CFG.num_folds
# num_features=len(feature_cols)
# num_targets=len(target_cols)

# # Averaging on multiple SEEDS
# SEED = np.arange(16)
# oof_seed = np.zeros((len(folds), len(target_cols)))
# predictions = np.zeros((len(test_noctl), len(target_cols)))
# losses_list = []
# # SEED = [[0,3],[1,1],[2,1],[3,2]]
# # for seed_fold, seed_run in tqdm(SEED):
# SEED = np.arange(16)
# for seed in tqdm(SEED):
#     seed_fold = seed // 4
#     seed_run = seed % 4

#     predictions_ = run_k_fold(NFOLDS, seed_fold, seed_run, display=2)
#     predictions += predictions_

# # FINAL CV LOGLOSS
# for col in target_cols:
#     test_noctl[col] = 0.0
# test_noctl[target_cols] = predictions / len(SEED)

In [61]:
# submission = sample_submission.drop(columns=target_cols)\
# .merge(test_noctl[['sig_id']+target_cols], on='sig_id', how='left')\
# .fillna(0.0).reset_index(drop=True)
# # sub.to_csv('submission.csv', index=False)
# # name_sub = 'submission.csv'
# # submission.to_csv(name_sub, index=False)
# # print(name_sub)
# print('TABNET 28nov')
# submission_TABNET_28nov = submission.copy()
# submission.head()

# BLENDING FINAL

In [62]:
# assert all(submission_TABNET_28nov.sig_id==submission_TABNET_30nov.sig_id)
# assert all(submission_TABNET_28nov.sig_id==submission_RESNET_30nov.sig_id)
# assert all(submission_TABNET_28nov.sig_id==submission_RESNET_28nov.sig_id)
# assert all(submission_TABNET_28nov.sig_id==submission_ANN_28nov.sig_id)
# assert all(submission_TABNET_28nov.sig_id==submission_ANN_30nov.sig_id)

In [63]:
# 229_blending_26nov
# names_models = ['../results/_NEW_83005_20nov_difffolds_1E3_4por4_newcalcfold_metrics/',
#                 '../results/_NEW_73002_model_TabNet_21nov_4por4_newcalcfold_metrics/',
#                 '../results/_NEW_83005_20nov_difffolds_1E3_4por4_newcalcfold_metrics_resnet_poly/']

# Optimization terminated successfully.
#          Current function value: 0.016602
#          Iterations: 100
#          Function evaluations: 205
# 0.016602136014725388 [0.38940193 0.32797063 0.2861724 ]
# blending_models([0.389,0.328,0.286])
# 0.016602141087921556



# names_models = ['../results/_NEW_83005_20nov_difffolds_1E3_4por4_newcalcfold_metrics/',
#                 '../results/_NEW_73002_model_TabNet_21nov_4por4_newcalcfold_metrics/',
#                 '../results/_NEW_83005_20nov_difffolds_1E3_4por4_newcalcfold_metrics_resnet_poly/',
#                '../results/_FINAL_ANN_CPS_STRAT/',
#                '../results/_FINAL_TABNET_CPS_STRAT/']
#0.01659238709151143
#[0.1075, 0.2098, 0.2107, 0.3352, 0.1414]



# w = [0.1075, 0.2098, 0.2107, 0.3352, 0.1414]



# names_models = ['../results/_NEW_83005_20nov_difffolds_1E3_4por4_newcalcfold_metrics/',
#                 '../results/_NEW_73002_model_TabNet_21nov_4por4_newcalcfold_metrics/',
#                 '../results/_NEW_83005_20nov_difffolds_1E3_4por4_newcalcfold_metrics_resnet_poly/',
#                    '../results/_FINAL_ANN_CPS_STRAT/',
#                    '../results/_FINAL_TABNET_CPS_STRAT/',
#                    '../results/_FINAL_RESNET_CPS_STRAT_v2/']
# Optimization terminated successfully.
#          Current function value: 0.016591
#          Iterations: 373
#          Function evaluations: 620
# 0.016590586038811306 [ 0.11163874  0.2195801   0.35596777  0.34734814  0.1496238  -0.17911374]

# w = [ 0.11163874,  0.2195801,   0.35596777,  0.34734814,  0.1496238,  -0.17911374] #[0.112, 0.220, 0.356, 0.347, 0.150, -0.179]




# FINAL

# names_models = [
#                 '../results/_NEW_83005_20nov_difffolds_1E3_4por4_newcalcfold_metrics/',
#                 '../results/_NEW_73002_model_TabNet_21nov_4por4_newcalcfold_metrics/',
#                 '../results/_NEW_83005_20nov_difffolds_1E3_4por4_newcalcfold_metrics_resnet_poly/',
                
#                '../results/_FINAL_ANN_CPS_STRAT_KMEANS7/',
#                '../results/_FINAL_TABNET_CPS_STRAT_KMEANS7/',
#                '../results/_FINAL_RESNET_CPS_STRAT_KMEANS7/']

# STRATIFIED BLENDING!!!!
# Optimization terminated successfully.
#          Current function value: 0.016590
#          Iterations: 402
#          Function evaluations: 674
# 0.01659034997712169 [0.0914673  0.18184113 0.14183788 0.3465606  0.15979273 0.08306089]

# w = [0.0914673, 0.18184113, 0.14183788, 0.3465606, 0.15979273, 0.08306089]

# submission = submission_ANN_28nov.copy()
# blending = (w[0]*submission_ANN_28nov.values[:,1:]) + \
#             (w[1]*submission_TABNET_28nov.values[:,1:]) + \
#             (w[2]*submission_RESNET_28nov.values[:,1:]) + \
#             (w[3]*submission_ANN_30nov.values[:,1:]) + \
#             (w[4]*submission_TABNET_30nov.values[:,1:]) + \
#             (w[5]*submission_RESNET_30nov.values[:,1:])


# STRATIFIED BLENDING!!!!
# Optimization terminated successfully.
#          Current function value: 0.016476
#          Iterations: 113
#          Function evaluations: 228
# 0.016476294101737145 [0.37846299 0.37115626 0.25789995]

#                '../results/_FINAL_ANN_CPS_STRAT_KMEANS7_SMOOTH_NO/',
#                '../results/_FINAL_TABNET_CPS_STRAT_KMEANS7_SMOOTH_NO/',
#                '../results/_FINAL_RESNET_CPS_STRAT_KMEANS7_SMOOTH_NO/']
            
# w = [0.37846299, 0.37115626, 0.25789995]

# w = [1.0/3, 1.0/3, 1.0/3]
# w = [ 0.48450186, -0.01045436,  0.52783731]



# STRATIFIED BLENDING!!!!
# Optimization terminated successfully.
#          Current function value: 0.016596
#          Iterations: 91
#          Function evaluations: 183
# 0.01659632784193557 [0.45209143 0.31521697 0.23694659]

w =  [0.45209143, 0.31521697, 0.23694659]

submission = submission_ANN_30nov.copy()
blending = (w[0]*submission_ANN_30nov.values[:,1:]) + \
            (w[1]*submission_TABNET_30nov.values[:,1:]) + \
            (w[2]*submission_RESNET_30nov.values[:,1:])



blending = blending.clip(0.0, 1.0)
submission.iloc[:,1:] = blending
submission.to_csv('submission.csv', index=False)

In [64]:
submission.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib